In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the number of input features, hidden units, and output classes
input_features = 10  # Example: 10 input features
hidden_units = 64    # Example: 64 hidden units
output_classes = 2   # Example: binary classification (2 classes)

# Create the sequential model
model = Sequential()

# Add the input layer and the hidden layer
model.add(Dense(hidden_units, input_shape=(input_features,), activation='relu'))


# Add the output layer
model.add(Dense(3, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


d:\Workspace\Repos\ParameterEstimator\env\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 899 (3.51 KB)

 Trainable params: 899 (3.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

model.compile(optimizer='adam', loss=custom_loss)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        # Define 3 custom neurons (weights)
        self.neuron1 = self.add_weight(name='neuron1', shape=(1,), initializer='uniform', trainable=True)
        self.neuron2 = self.add_weight(name='neuron2', shape=(1,), initializer='uniform', trainable=True)
        self.neuron3 = self.add_weight(name='neuron3', shape=(1,), initializer='uniform', trainable=True)

    def build(self, input_shape):
        # You can perform additional setup here if needed, and create more weights if necessary
        super(CustomLayer, self).build(input_shape)

    def call(self, inputs):
        # Define the computation of the layer using the custom neurons
        # Assuming 'inputs' is a tensor with shape (batch_size, 3) where each column corresponds to the output of a neuron
        output = self.neuron1 * inputs[:, 0] + self.neuron2 * inputs[:, 1] + self.neuron3 * inputs[:, 2]
        return output

# Example usage:
# Let's assume the input tensor has shape (batch_size, 3)
input_tensor = tf.keras.Input(shape=(3,))
custom_layer = CustomLayer()
output_tensor = custom_layer(input_tensor)

# Build the model
model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Summary of the model to check the custom layer
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_layer (CustomLayer)      │ (None)                 │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Define the input shape (number of features)
input_shape = (4,)  # For example, 4 features per sample

# Create a Sequential model and add a Dense layer
model = Sequential([
    Dense(units=3, activation='relu', input_shape=input_shape)  # Dense layer with 3 neurons
])

# Optionally, compile the model if you plan to train it
model.compile(optimizer='adam', loss='mse')

# Create a sample input tensor (e.g., a batch of 2 samples with 4 features each)
input_tensor = tf.constant([[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]])

# Get the forward pass output of the dense layer (also known as the layer's activation)
output_tensor = model(input_tensor)

# Print the output tensor
print(output_tensor)


tf.Tensor(
[[ 0.         0.         6.7167606]
 [ 0.         0.        17.390661 ]], shape=(2, 3), dtype=float32)


In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu",
                                             kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [ ]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="relu",
                                             kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [5]:
def negative_binomial_layer(x):
    """
    Lambda function for generating negative binomial parameters
    n and p from a Dense(2) output.
    Assumes tensorflow 2 backend.
    
    Usage
    -----
    outputs = Dense(2)(final_layer)
    distribution_outputs = Lambda(negative_binomial_layer)(outputs)
    
    Parameters
    ----------
    x : tf.Tensor
        output tensor of Dense layer
        
    Returns
    -------
    out_tensor : tf.Tensor
        
    """
    
    # Get the number of dimensions of the input
    num_dims = len(x.get_shape())
    
    # Separate the parameters
    mu, sd, alpha = tf.unstack(x, num=3, axis=-1)
    
    # Add one dimension to make the right shape
    mu = tf.expand_dims(mu, -1)
    sd = tf.expand_dims(sd, -1)
    alpha = tf.expand_dims(alpha, -1)
        
    # # Apply a softplus to make positive
    # n = tf.keras.activations.softplus(n)
    
    # # Apply a sigmoid activation to bound between 0 and 1
    # p = tf.keras.activations.sigmoid(p)
    
    # sd = tf.keras.activations.relu(sd)
    sd = tf.nn.relu(sd)

    # Join back together again
    out_tensor = tf.concat((mu, sd, alpha), axis=num_dims-1)

    return out_tensor

In [6]:
from scipy.stats import skewnorm

def negative_binomial_loss(y_true, y_pred, x):
    """
    Negative binomial loss function.
    Assumes tensorflow backend.
    
    Parameters
    ----------
    y_true : tf.Tensor
        Ground truth values of predicted variable.
    y_pred : tf.Tensor
        n and p values of predicted distribution.
        
    Returns
    -------
    nll : tf.Tensor
        Negative log likelihood.
    """

    # Separate the parameters
    mu, sd, alpha = tf.unstack(y_pred, num=3, axis=-1)
    
    # Add one dimension to make the right shape
    mu = tf.expand_dims(mu, -1)
    sd = tf.expand_dims(sd, -1)
    alpha = tf.expend_dims(alpha, -1)
    
    # Calculate the negative log likelihood
    L = (
        y_true - skewnorm.pdf(x, alpha, mu, sd)
    )           

    return L

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
class ResidualRegressor(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(64, activation="relu",
                                             kernel_initializer="he_normal")
        self.hidden2 = tf.keras.layers.Dense(3, activation="relu",
                                             kernel_initializer="he_normal")
        # self.paramLayer = negative_binomial_layer()
        # self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        Z = self.hidden2(Z)
        Z = Lamba(negative_binomial_layer)(Z)
        # for _ in range(1 + 3):
        #     Z = self.block1(Z)
        # Z = self.block2(Z)
        return self.out(Z)

In [ ]:
# from tensorflow.keras.layers import Input, Dense, Lambda
# from tensorflow.keras.models import Model

# Layer1 = Dense(1)
# output1 = Layer1(inputs)
# output2 = Layer2(output1)

# # Predict the parameters of a negative binomial distribution
# outputs = Dense(2)(output2)
# distribution_outputs = Lambda(negative_binomial_layer)(outputs)

# # Construct model
# model = Model(inputs=inputs, outputs=distribution_outputs)

In [7]:
from tensorflow.keras.optimizers import Adam

model = ResidualRegressor()
model.compile(loss=negative_binomial_loss, optimizer="adam")
